In [ ]:
import os
import dataclasses



import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

from google.protobuf import text_format
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))
tf.test.is_gpu_available(), tf.test.gpu_device_name()


In [ ]:

MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz'
MODEL_NAME = MODEL_URL.split('/')[-1].split('.')[0]
MODEL_NAME
CUSTOM_MODEL_NAME = 'my_ssd_net2'

In [ ]:
@dataclasses.dataclass
class Paths:
    tensorflow_dir: str = os.path.join('Tensorflow')
    models_dir: str = os.path.join(tensorflow_dir, 'models')
    custom_model_dir: str = os.path.join(models_dir, CUSTOM_MODEL_NAME)
    api_model_dir: str = os.path.join(tensorflow_dir, 'api_model')
    annotations_dir: str = os.path.join(tensorflow_dir, 'annotations')
    exported_models_dir: str = os.path.join(tensorflow_dir, 'exported_models')
    images_dir: str = os.path.join(tensorflow_dir, 'images')
    images_test_dir: str = os.path.join(images_dir, 'test')
    images_train_dir: str = os.path.join(images_dir, 'train')
    pre_trained_models_dir: str = os.path.join(tensorflow_dir, 'pre_trained_models')
    scriprs_dir: str = os.path.join(tensorflow_dir, 'scripts')
    records_dir: str = os.path.join(tensorflow_dir, 'records')
    checkpoints_dir: str = os.path.join(tensorflow_dir, 'checkpoints')

    
paths = Paths()

@dataclasses.dataclass
class Files:
    record_file: str = os.path.join(paths.records_dir, 'people.record')
    val_record_file: str = os.path.join(paths.records_dir, 'val_people.record')
    tf_record_generator_file: str = os.path.join(paths.scriprs_dir, 'my_tf_record.py')
    pipeline_config_file: str = os.path.join(paths.models_dir, CUSTOM_MODEL_NAME, 'pipeline.config')
    label_map_file: str = os.path.join(paths.annotations_dir, 'label_map.pbtxt')
    training_script_file: str = os.path.join(paths.api_model_dir,'research', 'object_detection', 'model_main_tf2.py')

files = Files()


In [ ]:
for i in dataclasses.asdict(paths).values():
    if not os.path.exists(i):
        os.mkdir(i)

In [ ]:
if not os.path.exists(os.path.join(paths.api_model_dir, 'research')):
    !git clone https://github.com/tensorflow/models.git $paths.api_model_dir
else:
    print('Api model already exists')

In [ ]:
try:
    import object_detection
    print('Object detection is already installed')
except ImportError:
    # Install object detection api
    !cd {paths.api_model_dir}/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install .

In [ ]:
!pip list | grep object-detection
!pip list | grep tensor


In [ ]:
# Train record
if not os.path.exists(files.record_file):
    !python {files.tf_record_generator_file} -a '{paths.annotations_dir}/train_bboxes.txt' -i '{paths.images_dir}' -o '{files.record_file}'
else:
    print('Train record already exists')

# Val record
if not os.path.exists(files.val_record_file):
    !python {files.tf_record_generator_file} -a '{paths.annotations_dir}/val_bboxes.txt' -i '{paths.images_dir}' -o '{files.val_record_file}'
else:
    print('Val record already exists')

In [ ]:
if not os.path.exists(os.path.join(paths.pre_trained_models_dir, MODEL_NAME)):
    !wget {MODEL_URL} -O {paths.pre_trained_models_dir}/{MODEL_NAME}.tar.gz
    !tar -xzvf {paths.pre_trained_models_dir}/{MODEL_NAME}.tar.gz -C {paths.pre_trained_models_dir}
else: 
    print('Pre-trained model already exists')


In [ ]:
!cp {paths.pre_trained_models_dir}/{MODEL_NAME}/pipeline.config {files.pipeline_config_file}

In [ ]:
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
config = config_util.get_configs_from_pipeline_file(files.pipeline_config_file)

pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files.pipeline_config_file, "r") as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)

pipeline_config.model.ssd.num_classes = 1
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(
    paths.pre_trained_models_dir, MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path = files.label_map_file
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [
    os.path.join(paths.records_dir, 'people.record')]
pipeline_config.eval_input_reader[0].label_map_path = files.label_map_file
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [
    os.path.join(paths.records_dir, 'val.record')]


config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(files.pipeline_config_file, "wb") as f:
    f.write(config_text)


In [ ]:
TRAINING_COMMAND = f'python {files.training_script_file} --model_dir {os.path.join(paths.models_dir, CUSTOM_MODEL_NAME)} --pipeline_config_path {files.pipeline_config_file}'
print(TRAINING_COMMAND)

In [ ]:
!{TRAINING_COMMAND}

In [ ]:
!cd /home/alex/Desktop && tensorboard --logdir .

# 8. Load Train Model From Checkpoint

In [ ]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util


In [ ]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(
    files.pipeline_config_file)
detection_model = model_builder.build(
    model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths.checkpoints_dir, 'ckpt-13')).expect_partial()


@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections


In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline


# 10. Real Time Detections from your Webcam

In [ ]:
!pip uninstall -y opencv-python
!pip install opencv-python

In [ ]:
!pip uninstall opencv-python-headless -y

In [ ]:
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print(cap.read())
while cap.isOpened():
    ret, frame = cap.read()
    image_np = np.array(frame)
    input_tensor = tf.convert_to_tensor(
        np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)

    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(
        np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_detections,
        detections['detection_boxes'],
        detections['detection_classes']+label_id_offset,
        detections['detection_scores'],
        1,
        use_normalized_coordinates=True,
        max_boxes_to_draw=5,
        min_score_thresh=.1,
        agnostic_mode=False)

    cv2.imshow('object detection',  cv2.resize(
        image_np_with_detections, (800, 600)))

    if cv2.waitKey(10) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break
